# Basic Example

This python notebook explains how to use the UP-JSPRIT library [Up-JSprit](www.google.com) for solving a Vehicle Routing Problem using the [Jsprit](https://github.com/graphhopper/jsprit) library where:
- locations are expressed in term of latitude and longitude
- the distance between two locations is evaluated using real data (provided through GraphHopper API) and used to calculate the cost matrix
- the real travel time between two locations is evaluated using real data (provided through Graphhopper API)  and used to calculate the cost matrix

The example wll go through the following main steps:
- Definition of a Vehicle Routing Problem using the unified-planning-library
- Call the ip-jsprit Solution Planner
- Show VRP Solution

**NOTE**: to run the example is needed to request an API_KEY from [GraphHopper](https://www.graphhopper.com/) website.
 parallel.

## Setup

The first step is the installation of following libraries and their dependencies from PyPi:
- the version 1.0.4 (the last currently available) of the up-jsprit library
- unified_planning library

In [ ]:
!pip install up-jsprit
!pip install unified_planning

## Imports

The libraries are imported for the correct running of the example

In [ ]:
import up_jsprit
from unified_planning.shortcuts import *
import unified_planning as up
from unified_planning import engines
from unified_planning.shortcuts import OneshotPlanner

## Problem Domain representation

In this example, a simple Vehicle Routing Problem (VRP) is specified where some items are located in a certain places and two Vehicles are in charge to pickup and deliver them in the final depot.

The example shows how to:
- define locations using latitude and longitude
- define characteristics of the key actions:
  - pickup:it represents the load of a good from a customer placed at a certain location to the vehicle
  - delivery: it represents the unload of a good from a vehicle to the Customer placed in a certain location
- define characteristics of the vehicle
  - load capacity: the maximum capacity that can be loaded on a vehicle
  - fixed cost: cost that does not depend on the travel distance
  - variable cost: cost that depend on the travel distance
- define characteristics of the item to be served:
   - time window availability
   - weight of each item
   - time needed for completing the service (delivery or pickup)
- manage the up-jpsrit library to solve the problem customizing the input parameters and the output, including the visualization on a georeferenced map.
 
Opportunely combining these elements it is possible to build complex examples like: 
- **CVRP** (Capacitated Vehicle Routing Problem): it is a VRP in which Vehicles have a limited carrying capacity of the goods that must be delivered
- **DVRP**P (Durative Vehicle Routing Problem): a classic VRP in which a duration is assigned to each action
- **VRPTW** W (Vehicle Routing Problem with Time Window): it is a generalization of the VRP where the service at any customer starts within a given time interval, called a time wind
- **MDVRP** (Multil Depot Vehicle Routing Problem): is a route optimization problem that involves selecting the most-effective route to deliver goods or services from multiple depots to a group of clients
- **VRPD** (Vehicle Routing with Pickups and Deliveries) is a VRP in which the possibility that customers return some commodities is contemplated. In VRPPD it’s needed to take into account that the goods that customers return to the delivery vehicle must fit into
- **VRPBTW** (Vehicle Routing Problem with Backhauls and Time Windows): involves the pickup and delivery of goods at different customer locations, with earliest and latest time deadlines, and varying demands. The demands are serviced using capacitated vehicles with limited route time. Moreover, all deliveries (linehauls) must be done before the pickups (backhauls)


#### Types

The step in defining the Problem Domain, referring to the [PDDL](https://planning.wiki/ref/pddl/domain) standard language, is to establish the set of uce a "Userneeded Type" to model the cs needed for defining the domain of problems that can be solved by the JSprit Library`. 

In [ ]:
######################################################################################
# DEFINE PROBLEM DOMAIN
######################################################################################

######################################################################################
# DEFINE USERTYPES
######################################################################################

Place = UserType("place")
VehicleType = UserType("VehicleType")
Locatable = UserType("Locatable")

Freight = UserType("Freight", Locatable)
Vehicle = UserType("Vehicle", Locatable)
Location = UserType("Location", Locatable)

#### Fluents
The second step in defining the Problem Domain, is to establish the set of "FLuents" needed to model the concepts needed to represent a problem

A set of numeric Fluents are defined to specify properties of:
- Services (pickup or delivery)
- Vehicle types

In [ ]:
######################################################################################
# DEFINE FLUENTS
######################################################################################

is_in = Fluent("is_in", BoolType(), f=Freight, l=Location)
is_at = Fluent("is_at", BoolType(), v=Vehicle, l=Location)
is_on = Fluent("is_on", BoolType(), f=Freight, v=Vehicle)
is_of = Fluent("is_of", BoolType(), v=Vehicle, vT=VehicleType)
using = Fluent("using", BoolType(), v=Vehicle)
t_earliest = Fluent("t_earliest", IntType(), f=Freight)
t_latest = Fluent("t_latest", IntType(), f=Freight)
x = Fluent('x', RealType(), l=Location)
y = Fluent('y', RealType(), l=Location)

# DEFINE FLUENTS FOR SERVICE - PICKUP OR DELIVERY
weight = Fluent("weight", RealType(), f=Freight)
loadTime = Fluent("loadTime", RealType(), f=Freight)

# DEFINE FLUENTS FOR VEHICLE TYPE
maxLoad = Fluent("maxLoad", RealType(), v=VehicleType)
fixedCost = Fluent("fixedCost", RealType(), vT=VehicleType)
variableCost = Fluent("variableCost", RealType(), vT=VehicleType)

#### Actions
The third step in defining the Problem Domain, is to establish the set of "Actions" needed to model the concepts needed to represent a problem

The actions allows to define different types of Vehicle routing Problems

The following representation is specific for the VRP problems that can be solved using the Jsprit library. It can be further enriched specifying other characteristics like the skills needed to perform a service.
This definition of Problem Domain is common to all the problems that can be solved interfacing the JSprit library with the up-jsprit library.

In [ ]:
######################################################################################
# DEFINE ACTIONS
######################################################################################

drive = InstantaneousAction("drive", v=Vehicle, l_from=Location, l_to=Location)
v = drive.parameter("v")
l_from = drive.parameter("l_from")
l_to = drive.parameter("l_to")

drive.add_precondition(is_at(v, l_from))

drive.add_effect(is_at(v, l_to), True)
drive.add_effect(is_at(v, l_from), False)

pickup = DurativeAction('pickup', f=Freight, l_from=Location, v=Vehicle, vt=VehicleType) 
f = pickup.parameter('f')
l_from = pickup.parameter('l_from')
v = pickup.parameter('v')
vt = pickup.parameter('vt')


pickup.set_fixed_duration(loadTime(f))
pickup.add_condition(StartTiming(), LE(weight(f), maxLoad(vt)))
#drive.add_precondition(is_in(f, l_from))

pickup.add_condition(StartTiming(), is_at(v, l_from))
pickup.add_condition(StartTiming(), is_in(f, l_from))
pickup.add_condition(StartTiming(), LE (t_earliest(f), StartTiming()))
pickup.add_condition(StartTiming(), GE (t_latest(f), StartTiming()))

#pickup.add_increase_effect(EndTiming(), value_weight(l), value_maxLoad(mL))
pickup.add_increase_effect(EndTiming(), weight(f), maxLoad(vt))
pickup.add_effect(EndTiming(), is_in(f, l_from), False)
pickup.add_effect(EndTiming(), is_on(f, v), True)


delivery = DurativeAction('delivery', f=Freight, l_to=Location, v=Vehicle, vt=VehicleType)
f = delivery.parameter('f')
l_to = delivery.parameter('l_to')
v = delivery.parameter('v')
vt = delivery.parameter('vt')


delivery.set_fixed_duration(loadTime(f))
#drive.add_precondition(is_in(f, l_from))

delivery.add_condition(StartTiming(), is_at(v, l_to))
delivery.add_condition(StartTiming(), not (is_in(f, l_to)))
delivery.add_condition(StartTiming(), LE (t_earliest(f), StartTiming()))
delivery.add_condition(StartTiming(), GE (t_latest(f), StartTiming()))
#delivery.add_condition(StartTiming(), available(f, l_to, t_earliest, t_lastest))

delivery.add_decrease_effect(EndTiming(), weight(f), maxLoad(vt))
delivery.add_effect(EndTiming(), is_in(f, l_to), True)
delivery.add_effect(EndTiming(), is_on(f, v), False)

## Problem Representation

### Problem Initialization
Once defined the problem domain, the VRP can be initialized providing a name a adding fluents and action. Then, the VRP can be specified in detalis providing initial values and final goals according to steps described as follows

In [ ]:
######################################################################################
# DEFINE PROBLEM NAME
######################################################################################

problem = Problem('VehicleRoutingProblem')

######################################################################################
# INITIALIZE FLUENTS AND ACTIONS
######################################################################################

#problem.add_fluent(at)
problem.add_fluent(is_in, default_initial_value=False)
problem.add_fluent(is_on, default_initial_value=False)
problem.add_fluent(is_at, default_initial_value=False)
problem.add_fluent(t_earliest, default_initial_value=0)
problem.add_fluent(t_latest, default_initial_value=100000)

problem.add_fluent(weight, default_initial_value=0)
problem.add_fluent(maxLoad, default_initial_value=0)

problem.add_action(drive)
problem.add_action(delivery)
problem.add_action(pickup)

### STEP 1 - Vehicle Type Definition
The scope is to define the set of Vehicle Type used in the VRP. In a second step the Vehicle Type is assigned to one, more or all the Vehicle foreseen in the VRP. For sake of simplicity, in this example, only a Vehicle Type is considered.

In [ ]:
######################################################################################
# STEP 1 - DEFINE VEHICLE TYPE
######################################################################################

vt1 = Object("vt1", VehicleType)
problem.add_object(vt1)

### STEP 2 - Vehicle Definition
The scope is to define the set of Vehicle used in the VRP. For sake of simplicity, in this example, only three Vehicles are considered. Note that the solution can involve only a subset of vehicles if it is less expensive solution.

In [ ]:
######################################################################################
# STEP 2 - DEFINE VEHICLE NAME
######################################################################################

v1 = Object("v1", Vehicle)
problem.add_object(v1)
v2 = Object("v2", Vehicle)
problem.add_object(v2)
v3 = Object("v3", Vehicle)
problem.add_object(v3)

### STEP 3 - Location Definition
The scope is to define the locations where are the items to be delivered/picked-up and the depots where the vehicles are supposed to start and end the services. In this example, the initial and final destinations (depots) of all the vehicles are the locations named "l0" and "l101".
Note that Vehicles can have different depots.

In [ ]:
######################################################################################
# STEP 3 - DEFINE LOCATIONS - COORDINATES
######################################################################################

l0 = Object("l0", Location) #Initial Depot for all the Vehicles
problem.add_object(l0)
l1 = Object("l1", Location)
problem.add_object(l1)
l2 = Object("l2", Location)
problem.add_object(l2)
l3 = Object("l3", Location)
problem.add_object(l3)
l4 = Object("l4", Location)
problem.add_object(l4)
l5 = Object("l5", Location)
problem.add_object(l5)
l101 = Object("l101", Location)  #Final Depot for all the Vehicles
problem.add_object(l101)

### STEP 4 - Freight Definition
The scope is to define the freights that have to be delivered and picked up. in the next step they will be assigned to a specific locations and further characterized with additional properties.
In this example only 4 Freight are defined.

In [ ]:
######################################################################################
# STEP 4 - DEFINE FREIGHT  PARTICIPATING TO DELIVERY OR PICKUP
######################################################################################

f1 = Object("f1", Freight)
problem.add_object(f1)
f2 = Object("f2", Freight)
problem.add_object(f2)
f3 = Object("f3", Freight)
problem.add_object(f3)
f4 = Object("f4", Freight)
problem.add_object(f4)


### STEP 5 - Locations Coordinates Initialization
The scope is to assign a value to the x and y coordinates of the Location entity. If the VRP problem is defined with georeferenced data, these values representes the latitude and longitude of the Location.
The number is defined as an integer with only 4 decimal numbers. This assumption shall be carefully take into account when the User want to modify this data. 

In [ ]:
######################################################################################
# STEP 5 - INITIALIZE LOCATION COORDINATES
######################################################################################

problem.set_initial_value(x(l0), 419028)
problem.set_initial_value(y(l0), 124964)
problem.set_initial_value(x(l1), 418986)
problem.set_initial_value(y(l1), 124739)
problem.set_initial_value(x(l2), 419009)
problem.set_initial_value(y(l2), 125025)
problem.set_initial_value(x(l3), 419183)
problem.set_initial_value(y(l3), 124667)
problem.set_initial_value(x(l4), 418902)
problem.set_initial_value(y(l4), 124923)
problem.set_initial_value(x(l5), 418833)
problem.set_initial_value(y(l5), 124541)
problem.set_initial_value(x(l101), 419361) 
problem.set_initial_value(y(l101), 124690)

### STEP 6 - Service Property Initialization
The scope is to assign a value to parameter that characterize a service (delivery or pickup). In the JSprit terminology, in addition to the delivery and pickup serivce, the shipment service is defined when an item is loaded (picked-up) and unloaded (delivered) in two different locations that are not the depots.
For each freight the User can define following parameters:
- **weight** that is summed or subtracted to the maximum capacity of the vechicle when it is picked-up or delivered
- **loadTime** that is the time needed to perform the load or unload of the freight in the location
- **time windows** of availability when the service can be performed

In [ ]:
######################################################################################
# STEP 6 DEFINE PROPERTY OF THE SHIPMENT
######################################################################################

problem.set_initial_value(weight(f1), 10)
problem.set_initial_value(loadTime(f1), 90)
problem.set_initial_value(t_earliest(f1), 283000)
problem.set_initial_value(t_latest(f1), 883000)

problem.set_initial_value(weight(f2), 90)
problem.set_initial_value(loadTime(f2), 90)
problem.set_initial_value(t_earliest(f2), 15000)
problem.set_initial_value(t_latest(f2), 716000)

problem.set_initial_value(weight(f3), 40)
problem.set_initial_value(loadTime(f3), 90)
problem.set_initial_value(t_earliest(f3), 87000)
problem.set_initial_value(t_latest(f3), 900000)

problem.set_initial_value(weight(f4), 10)
problem.set_initial_value(loadTime(f4), 90)
problem.set_initial_value(t_earliest(f4), 65000)
problem.set_initial_value(t_latest(f4), 716000)

### STEP 7 - Vehicle Type Properties Initialization
The scope is to assign a value to the following properties of Vehicle Type:
- **maxLoad**: the maximum load alowed for the Vehicle Type
- **fixedCosts**: it is fixed costs for each service
- **variablecosts**: it is the costs per distance/time unit considered for each service

In the Jsprit library these are a property of the Vehicle Type. Once assigned a Vehicle Type to the Vehicle it is applied to each vehicle.parameter that characterize a service (delivery or pickup). In the JSprit terminology, in addition to the delivery and pickup serivce, the shipment service is defined when an item.
In this example only one Vehicle Type is defined and its properties defined.

In [ ]:
######################################################################################
# STEP 7 - INITIALIZE MAX CAPACITY, FIXED AND VARIABLE COSTS OF VEHICLES TYPE
######################################################################################

problem.set_initial_value(maxLoad(vt1),1000)
problem.set_initial_value(fixedCost(vt1),22)
problem.set_initial_value(variableCost(vt1),3)

### STEP 8 - Vehicle Type Assignment
The scope is to assign the defined Vehicle Types to the Vehicles defined for the VRP. In this example all the Vehicle are of the same Type 

In [ ]:
######################################################################################
# STEP 8 - ASSIGN VEHICLE TYPE TO VEHICLE
######################################################################################

problem.set_initial_value(is_of(v1, vt1), True)
problem.set_initial_value(is_of(v2, vt1), True)
problem.set_initial_value(is_of(v3, vt1), True)

### STEP 9 - Assignment of Initial Location of Shipments
The scope is to assign to each freight the location where they have to picked up. In this example all the freight are picked-up in differen locations and delivered to a the same depot. If the User want to define a different VRP where some freight are already in a depot, he shall simply assigne the freight to the depot location.

In [ ]:
######################################################################################
# STEP 9 - DEFINE INITIAL LOCATION OF SHIPMENTS - PICKUP
######################################################################################

problem.set_initial_value(is_in(f1, l1), True)
problem.set_initial_value(is_in(f2, l2), True)
problem.set_initial_value(is_in(f3, l3), True)
problem.set_initial_value(is_in(f4, l4), True)

### STEP 10 - Assignment of Initial Location of Vehicles
The scope is to assign to each Vehicle the location where to start the services (depot). In this example all the Vehicles are assigned to the same depot (l0).

In [ ]:
######################################################################################
# STEP 10 - DEFINE INITIAL LOCATION OF VEHICLES
######################################################################################
problem.set_initial_value(is_at(v1, l0), True)
problem.set_initial_value(is_at(v2, l0), True)
problem.set_initial_value(is_at(v3, l0), True)

### STEP 11 - Assignment of Final Location of Shipments
The scope is to assign to each Freight the location where they shall be delivered. In this example the Freight f1, f2, f3 are delivered to the depot (using the Jsprit notation, this case correspond to a pickup service), f4 is delivered to the location l5 (using the Jsprit notation, this case correspond to a shipment service).

In [ ]:
######################################################################################
# STEP 11 - DEFINE FINAL LOCATION OF SHIPMENT - DELIVERY
######################################################################################

problem.add_goal(is_in(f1, l101))
problem.add_goal(is_in(f2, l101))
problem.add_goal(is_in(f3, l101))
problem.add_goal(is_in(f4, l5))

### STEP 12 - Assignment of Final Location of Vehicles
The scope is to assign to each Vehicle the location where to finish the services (depot). In this example all the Vehicles are assigned to the same final depot (l101).

In [ ]:
######################################################################################
# STEP 12 - DEFINE FINAL LOCATION OF VEHICLES - DEPOT
######################################################################################

problem.add_goal(is_at(v1, l101))
problem.add_goal(is_at(v2, l101))
problem.add_goal(is_at(v3, l101))

print(problem)

### STEP 13 - Save the Problem
The scope is to save the problem in a text file that can be parsed and injested to the Jsprit engine. The name and the path of the file can be changed and, by default is save in the ./ directory.

In [ ]:
######################################################################################
# STEP 13 - Save the Problem
######################################################################################
with open("./parsed_problem.txt", 'w') as file:
    file.write(str(problem))

### STEP 14 - Setup the Unified Planning Library
The scope is to setup the Jsprit engine to solve the probelm using the up-jsprit library.

In [ ]:
######################################################################################
# STEP 14 - Setup the UPL Environment
######################################################################################
env = up.environment.get_environment()
env.factory.add_engine('jspritplanner', 'up_jsprit', 'JSpritSolver')

### STEP 15 - Save the Problem
The scope is to solve the problem using the JSprit engine. The solve the problem it is possible to define the following parameters:
- **max_iterations**: the maximum number of iteration to find the solution
- **output_dir**: the directory where output are saved
- **problem_filename**: the name of the text file where is saved the problem. The name shall be the same used in the step 13
- **solution_filename**: the name of the filename containing the best solution found by the JSprit engine. The file is saved in the output directory previously specified
- **geocoordinates**: sit specifies if the solution must be found using georeferenced data. This implies that the cost matrix is evaluated using real route obtained through the GraphHopper API
- **api**: API for GraphHopper site
- **debug**: it defines if additional info are printout dirung the execution for debug scopes. Default value is False,
- **view** : it defines if a viewer showing the graph of the solution shall be prompted out. For Colab execution it must be set to False. Default Value True

At the end of the process, the solution is evaluated as:
- **SOLVED_OPTIMALLY** if all services requested are planned
- **SOLVED_SATISFICING** if less than 10% of requested services are not planned
- **UNSOLVED** if more than 10% of requested services are not planned


In [ ]:
######################################################################################
# STEP 15 - Solve the Problem
######################################################################################

with OneshotPlanner(name='jspritplanner', params = {'max_iterations' : 2000, 'output_dir' : '.\\', 'problem_filename' : 'parsed_problem.txt', 'solution_filename': 'BestSolution.txt', 'geocoordinates': True, 'debug': True, 'api' : '36d16024-8b24-4091-bb89-7adef7632a20','view' : False}) as p:
    result = p.solve(problem)
    if result.status == up.engines.PlanGenerationResultStatus.SOLVED_OPTIMALLY:
        print(f'{p.name} found a valid plan!')
        print(result.plan)
    elif result.status == up.engines.PlanGenerationResultStatus.SOLVED_SATISFICING:
        print(f'{p.name} found a satisficing plan!')
        print(result.plan)
    else:
        print('No satisficin plan found!')

### STEP 16 - View the Results

The up-jSprit engine produce a set of files as ouput that can viewed by the User after the completion of the run:
1) **routes_map.html** is an html page that show the routes planned
2) **BestSolution.txt** is the solution. For each route it is specified
   - the type of activity (start, pickup, deliver, end)
   - the arrival time
   - the departure time
   - costs
3) **SolutionPlot.png** show the solution on a simple graph. It is automatically produced by the JSprit library
4) **solution_progress.png** shows the progress of the solution as function of the iterations

  Other text files are produced when parsing the problem:
  - goals.txt contains the info related to the goals to achieve. This file is further parsed by the up-jpsrit library and injested to th Jsprit engine solver
  - initial values.txt contains the info related to the intial values of the VRP. This file is further parsed by the up-jpsrit library and injested to th Jsprit engine solver